In [ ]:
import pandas as pd

import spacy
from spacy import displacy

from spacy.matcher import Matcher
from spacy.matcher import DependencyMatcher
from spacy.tokens import Token

Token.set_extension("ignore", default=False, force=True)

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

nlp = spacy.load("en_core_web_trf")

In [2]:
sentences = pd.read_excel('../data/demo_data_augmented.xlsx')

In [5]:
dataset = list(sentences[["Original Text EN", "Tags/Kategorie"]].sample(frac=1).itertuples(index=False, name=None))
train_data = dataset[:3000]
dev_data = dataset[3000:3200]
test_data = dataset[3200:3375]
print(f"Total: {len(dataset)} - Train:  {len(train_data)} - Dev: {len(dev_data)} - Test: {len(test_data)}")

Total: 3375 - Train:  3000 - Dev: 200 - Test: 175


In [6]:
labels_test = sentences['Tags/Kategorie'].apply(lambda x : x.split(","))

all_labels = []
for li in labels_test:
    for label in li:
        label = label.strip()
        all_labels.append(label)

all_labels = set(all_labels)
print(all_labels)
all_labels = list(all_labels)
all_labels


{'ERP', 'Python', 'API', 'DWH', 'Zertifikate', 'SQL', 'Kubernetes', 'Machine Learning', 'SAP', 'Grafana', 'CRM', 'HR', 'Allgemeine Definitionen', 'MLFlow', 'DeepL'}


['ERP',
 'Python',
 'API',
 'DWH',
 'Zertifikate',
 'SQL',
 'Kubernetes',
 'Machine Learning',
 'SAP',
 'Grafana',
 'CRM',
 'HR',
 'Allgemeine Definitionen',
 'MLFlow',
 'DeepL']

In [7]:
def convert(data, outfile):
    db = spacy.tokens.DocBin()
    docs = []
    for doc, label in nlp.pipe(data, as_tuples=True):
        for l in all_labels:
            if l in label: 
                doc.cats[l] = 1
            else:
                doc.cats[l] = 0
        db.add(doc)

        print(doc.cats)
    
    db.to_disk(outfile)

In [8]:
convert(train_data, "./train.spacy")
convert(dev_data, "./dev.spacy")
convert(test_data, "./test.spacy")

{'ERP': 1, 'Python': 0, 'API': 0, 'DWH': 1, 'Zertifikate': 0, 'SQL': 0, 'Kubernetes': 0, 'Machine Learning': 0, 'SAP': 0, 'Grafana': 0, 'CRM': 0, 'HR': 0, 'Allgemeine Definitionen': 0, 'MLFlow': 0, 'DeepL': 0}
{'ERP': 0, 'Python': 0, 'API': 0, 'DWH': 0, 'Zertifikate': 0, 'SQL': 0, 'Kubernetes': 1, 'Machine Learning': 0, 'SAP': 0, 'Grafana': 0, 'CRM': 0, 'HR': 0, 'Allgemeine Definitionen': 0, 'MLFlow': 0, 'DeepL': 0}
{'ERP': 0, 'Python': 0, 'API': 0, 'DWH': 0, 'Zertifikate': 0, 'SQL': 0, 'Kubernetes': 0, 'Machine Learning': 0, 'SAP': 0, 'Grafana': 0, 'CRM': 1, 'HR': 0, 'Allgemeine Definitionen': 1, 'MLFlow': 0, 'DeepL': 0}
{'ERP': 0, 'Python': 0, 'API': 0, 'DWH': 0, 'Zertifikate': 0, 'SQL': 0, 'Kubernetes': 0, 'Machine Learning': 0, 'SAP': 0, 'Grafana': 0, 'CRM': 0, 'HR': 1, 'Allgemeine Definitionen': 0, 'MLFlow': 0, 'DeepL': 0}
{'ERP': 1, 'Python': 0, 'API': 0, 'DWH': 1, 'Zertifikate': 0, 'SQL': 0, 'Kubernetes': 0, 'Machine Learning': 0, 'SAP': 0, 'Grafana': 0, 'CRM': 0, 'HR': 0, 'Allg

ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'int'>

In [ ]:
!python -m spacy init config --lang pt --pipeline textcat_multilabel --optimize efficiency --force config.cfg

In [ ]:
!python -m spacy train config.cfg --paths.train ./train.spacy  --paths.dev ./dev.spacy --output model --verbose

In [ ]:
!python -m spacy evaluate ./model/model-best/ ./test.spacy